In [1]:
import pandas as pd
import jieba

In [2]:
sms_data = pd.read_csv('data/80w.txt', sep='\t', header=None)
sms_data.head()

,0,1,2
0,1,0,商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一
1,2,1,南口阿玛施新春第一批限量春装到店啦   春暖花开淑女裙、冰蓝色公主衫 ...
2,3,0,带给我们大常州一场壮观的视觉盛宴
3,4,0,有原因不明的泌尿系统结石等
4,5,0,23年从盐城拉回来的麻麻的嫁妆


第一列为原始序号，第二列为短信的分类，0表示正常短信，1表示垃圾短信，第三列就是短信的正文。

In [3]:
print(sms_data.shape)

(754843, 3)


In [4]:
# stop words
stop_words = pd.read_csv('stopWord.txt', header=None)
stop_words.head()

,0
0,、
1,，
2,。
3,？
4,喔


In [5]:
STOP_WORDS = frozenset((stop_words[0].values))
def participle(text):
    seg_list = jieba.cut(text)
    results = set(seg_list) - STOP_WORDS
    return ' '.join(results)

In [6]:
sms_data['participle'] = sms_data[2].apply(participle)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.166 seconds.
Prefix dict has been built succesfully.


In [7]:
sms_data.head()

,0,1,2,participle
0,1,0,商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一,和 商业秘密 是 那 的 垄断 地位 商业价值 秘密性 维系 之一 前提条件 其
1,2,1,南口阿玛施新春第一批限量春装到店啦   春暖花开淑女裙、冰蓝色公主衫 ...,裙 蓝色 到 春装 气质 粉小  阿玛施 皇 春暖花开 店 淑女 冰 公主 啦 第一批...
2,3,0,带给我们大常州一场壮观的视觉盛宴,一场 常州 带给 大 盛宴 的 壮观 我们 视觉
3,4,0,有原因不明的泌尿系统结石等,结石 的 不明 有 原因 泌尿系统 等
4,5,0,23年从盐城拉回来的麻麻的嫁妆,拉回来 年 的 盐城 嫁妆 从 麻麻 23


In [8]:
X = sms_data['participle'].values
Y = sms_data[1].values

In [9]:
print(Y)

[0 1 0 ... 0 0 0]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import shuffle

random_seed = 17
X_cross_train, X_cross_test, Y_cross_train, Y_cross_test = train_test_split(X, Y, test_size=0.3, random_state=random_seed)
#X_cross_train, y_cross_train = shuffle(X_cross_train, Y_cross_train, random_state=random_seed)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

vectorizer = CountVectorizer()
X_train_termcounts = vectorizer.fit_transform(X_cross_train)

tfidf_tansformer = TfidfTransformer()
X_train_tfidf = tfidf_tansformer.fit_transform(X_train_termcounts)

In [12]:
## Transform persistence
import pickle

# Save to file in the current working directory
pkl_filename = "vectorizer.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(vectorizer, file)
    
pkl_filename = "transformer.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(tfidf_tansformer, file)

In [13]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

classifier = MultinomialNB().fit(X_train_tfidf, Y_cross_train)

In [14]:
## Model persistence

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(classifier, file)

In [15]:
X_test_termcounts = vectorizer.transform(X_cross_test)
X_test_tfidf = tfidf_tansformer.transform(X_test_termcounts)

predicted_categories = classifier.predict(X_test_tfidf)

In [16]:
print(X_cross_test)

['细胞 照片 x 好 的 了 整理 自己 今天' '细胞 我 手胃 都 不 的 我心 开心 汗毛 在 浑身 脑子 一个 每'
 '近朱者赤 哦 近墨者黑 叫做 就 这' ... '： 杭州 有限公司 卖方 海通 营业部 股份 证券 为 文化路'
 '完全 得 好 不想 没 在 了 睡觉 飞机 睡 上' '和 神秘 讲述 大盗 了 美女 一个 小偷 三个 影片']


In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_cross_test, predicted_categories)

0.983744971362711

In [18]:
category_map = {
    0: 'normal',
    1: 'spam'
}

for sentence, category, real in zip(X_cross_test[:100], predicted_categories[:100], Y_cross_test[:100]):
    print('\nsms_content: ', sentence, '\nPredicted: ', category_map[category], '\ntrue value: ', category_map[real])


sms_content:  细胞 照片 x 好 的 了 整理 自己 今天 
Predicted:  normal 
true value:  normal

sms_content:  细胞 我 手胃 都 不 的 我心 开心 汗毛 在 浑身 脑子 一个 每 
Predicted:  normal 
true value:  normal

sms_content:  近朱者赤 哦 近墨者黑 叫做 就 这 
Predicted:  normal 
true value:  normal

sms_content:  怕 我 是 举报 夜停 公车私用 警车 小区 大概 
Predicted:  normal 
true value:  normal

sms_content:  发生爆炸 笔记本电脑 的 将 突然 联想 自己 桌子 烧焦 
Predicted:  normal 
true value:  normal

sms_content:  MTS 纹眉 纹 管理 水光针 咨询 项目 半 可 皮肤 美白针 眼线 永久 店内 纹唇 等 
Predicted:  normal 
true value:  normal

sms_content:  带动 有所 保险 下 的 反弹 权重股 等 
Predicted:  normal 
true value:  normal

sms_content:  我 手机 半路上 提示 了 延误 飞机 
Predicted:  normal 
true value:  normal

sms_content:  免受 保护 侵害 永久 电脑 XP 承诺 
Predicted:  normal 
true value:  normal

sms_content:  欢乐 多余 花千骨 后面 的 觉得 有 一直 不该 小说 
Predicted:  normal 
true value:  normal

sms_content:  一支 韩国 美女 几乎 人手 
Predicted:  normal 
true value:  normal

sms_content:  安详 丝 民族村 生活 背心 + 吊带 慢 半截 
Predicted:  normal 
true value:  normal

sms_content:  